In [1]:
import pandas as pd
import numpy as np

In [2]:
load_dir = "C:\\Users\\jandr\\code_projects\\MVSK\\assets\\2022-01-12T18-13-20.768\\"

In [5]:
df = pd.read_csv(load_dir+"parameters.csv", header=None, delimiter = '|')

In [6]:
f_1_max = float(df[0][1])
f_2_min = float(df[1][1])

In [9]:
# Import data
means_vec = np.array(pd.read_csv(load_dir+"means_vector.csv", header=None,delimiter = '|'))[0]
cov_mat   = np.array(pd.read_csv(load_dir+"covariance_matrix.csv", header=None,delimiter = '|'))
skew_mat  = np.array(pd.read_csv(load_dir+"skewness_matrix.csv", header=None,delimiter = '|'))
kur_mat   = np.array(pd.read_csv(load_dir+"kurtosis_matrix.csv", header=None,delimiter = '|'))               

In [10]:
n = means_vec.shape[0]
lam = np.ones(4)*0.25

In [11]:
f_3_hat_estimates = pd.read_csv(load_dir+"f₃ₘₐₓ_nl_df.csv", header="infer",delimiter = '|')
f_4_hat_estimates = pd.read_csv(load_dir+"f₄ₘᵢₙ_nl_df.csv", header="infer",delimiter = '|')

In [12]:
f_3_hat = f_3_hat_estimates['obj_val'][5]
f_4_hat = f_4_hat_estimates['obj_val'][5]

In [14]:
import sageopt as so
z = so.standard_sig_monomials(n+4)
y = z[:n]
x = z[n:]
f = x[0]**lam[0] + x[1]**lam[1] + x[2]**lam[2] + x[3]**lam[3]
mu_y        =  np.dot(means_vec, y) 
yT_cov_y    =  np.dot(np.dot(np.transpose(y), cov_mat), np.reshape(y,(n,1)))[0] 
yT_skew_yy  =  np.dot(np.dot(y, skew_mat), np.reshape(np.kron(y,y),(n**2,1)))[0] 
yyT_skew_yy =  np.dot(np.dot(np.transpose(np.kron(y,y)), kur_mat), np.reshape(np.kron(y,y),(n**2,1)))[0] 

In [15]:
eqs = [ sum(y) - 1]
gts = [x[0] - (1 - (mu_y / f_1_max) ),
       x[1] - ((yT_cov_y / f_2_min) - 1),
        x[2] - (1 - (yT_skew_yy / f_3_hat)),
        x[3] - ((yyT_skew_yy / f_4_hat) - 1),
        f_3_hat - yT_skew_yy,
        yyT_skew_yy - f_4_hat 
       ]
bounds = []

In [16]:
X = so.infer_domain(f, bounds, [])
prob = so.sig_constrained_relaxation(f, gts, eqs, X)
prob.solve(verbose=True)



Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2358            
  Cones                  : 31766           
  Scalar variables       : 95524           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.02            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.05    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimiz

('solved', 2.2951933979500848e-09)

In [ ]:
print(prob.value)

In [75]:
solrec = so.sig_solrec(prob)

In [74]:
solrec

[]

In [ ]:
prob_rel = so.sig_constrained_relaxation(f, gts, eqs, X, ell=1)

In [18]:
prob_rel.solve(verbose=True)



Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 51809           
  Cones                  : 19391806        
  Scalar variables       : 58180145        
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 1.44            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 32.94   
MOSEK error 1051: Out of space.
Optimizer terminated. Time: 57.92   



Error: rescode.err_space(1051): Out of space.